In [1]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
import warnings
%pylab inline
warnings.filterwarnings('ignore')


Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv(r'C:\Users\admcc\Desktop\aac_shelter_outcomes.csv')
df.columns.values

array(['age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
       'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype',
       'outcome_type', 'sex_upon_outcome'], dtype=object)

In [3]:
df

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male
6,1 year,A693700,Other,Squirrel Mix,Tan,2013-12-13T00:00:00,2014-12-13T12:20:00,2014-12-13T12:20:00,NaN,Suffering,Euthanasia,Unknown
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23T00:00:00,2014-12-08T15:55:00,2014-12-08T15:55:00,*Ella,Partner,Transfer,Spayed Female
8,1 month,A685067,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,2014-08-14T18:45:00,Lucy,NaN,Adoption,Intact Female
9,3 months,A678580,Cat,Domestic Shorthair Mix,White/Black,2014-03-26T00:00:00,2014-06-29T17:45:00,2014-06-29T17:45:00,*Frida,Offsite,Adoption,Spayed Female


In [5]:
df['breed'].value_counts

<bound method Series.value_counts of 0                      Domestic Shorthair Mix
1                                  Beagle Mix
2                                    Pit Bull
3                     Miniature Schnauzer Mix
4                                     Bat Mix
5                              Leonberger Mix
6                                Squirrel Mix
7                     Chihuahua Shorthair Mix
8                      Domestic Shorthair Mix
9                      Domestic Shorthair Mix
10                   Domestic Medium Hair Mix
11                     Papillon/Border Collie
12             Chihuahua Shorthair/Pomeranian
13       Miniature Schnauzer/Miniature Poodle
14                     Labrador Retriever Mix
15                            Rat Terrier Mix
16                               Pit Bull Mix
17                     Domestic Shorthair Mix
18         German Shepherd/Labrador Retriever
19                     Domestic Shorthair Mix
20                     Domestic Shorthair M

In [37]:
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], format='%Y-%m-%d')
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d')

df['days_alive'] = df['datetime'].sub(df['date_of_birth'], axis=0)
df['days_alive'] = df['days_alive'].dt.days
df.describe()

,days_alive,target
count,78256.000000,78256.000000
mean,817.039946,0.423086
std,1077.094186,0.494052
min,0.000000,0.000000
25%,95.000000,0.000000
50%,371.000000,0.000000
75%,1096.000000,1.000000
max,9137.000000,1.000000


In [38]:
df['target'] = 0.0
mask = (df['outcome_type'] == 'Adoption')
df['target'][mask] = 1.0
df['target'].value_counts()

0.0    45147
1.0    33109
Name: target, dtype: int64

In [39]:

df['age_binned'] = pd.qcut(df['days_alive'], 5)
print df['age_binned'][:5]

0      (-0.001, 78.0]
1      (306.0, 669.0]
2      (306.0, 669.0]
3    (1222.0, 9137.0]
4       (78.0, 306.0]
Name: age_binned, dtype: category
Categories (5, interval[float64]): [(-0.001, 78.0] < (78.0, 306.0] < (306.0, 669.0] < (669.0, 1222.0] < (1222.0, 9137.0]]


In [40]:
categorical_columns = ['age_binned', 'animal_type', 'sex_upon_outcome']
df_dummies = pd.get_dummies(df[categorical_columns],
                            prefix=categorical_columns,
                            columns=categorical_columns)
dummy_column_names = df_dummies.columns.values
dummy_column_names[:10]

array(['age_binned_(-0.001, 78.0]', 'age_binned_(78.0, 306.0]',
       'age_binned_(306.0, 669.0]', 'age_binned_(669.0, 1222.0]',
       'age_binned_(1222.0, 9137.0]', 'animal_type_0', 'animal_type_Bird',
       'animal_type_Cat', 'animal_type_Dog', 'animal_type_Livestock'], dtype=object)

In [41]:
df2 = pd.concat([df, df_dummies], axis=1)

In [42]:
formula = 'target ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in dummy_column_names]))
print formula

target ~ 0 + Q("age_binned_(-0.001, 78.0]") + Q("age_binned_(78.0, 306.0]") + Q("age_binned_(306.0, 669.0]") + Q("age_binned_(669.0, 1222.0]") + Q("age_binned_(1222.0, 9137.0]") + Q("animal_type_0") + Q("animal_type_Bird") + Q("animal_type_Cat") + Q("animal_type_Dog") + Q("animal_type_Livestock") + Q("animal_type_Other") + Q("sex_upon_outcome_0") + Q("sex_upon_outcome_Intact Female") + Q("sex_upon_outcome_Intact Male") + Q("sex_upon_outcome_Neutered Male") + Q("sex_upon_outcome_Spayed Female") + Q("sex_upon_outcome_Unknown")


In [43]:
Y, X = dmatrices(formula, df2, return_type='dataframe')

In [44]:
y = Y['target'].values
y[:10]

array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.])

In [45]:
from sklearn import naive_bayes
model = naive_bayes.MultinomialNB()

In [46]:
model.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [36]:
print 'Prediction'
print model.predict(X[:10])
print 'Actual'
print y[:10]

Prediction
[ 0.  1.  1.  0.  0.  0.  0.  1.  0.  1.]
Actual
[ 0.  0.  1.  0.  0.  0.  0.  0.  1.  1.]


In [47]:
from sklearn import metrics

prediction_train = model.predict(X)
print metrics.accuracy_score(y, prediction_train)

0.736211919853


In [48]:
feature_importances = abs(model.feature_log_prob_[1] - model.feature_log_prob_[0])
feature_importances

array([ 0.07650386,  0.7717812 ,  0.00946049,  0.25508321,  0.46919388,
        1.88716629,  0.34317228,  0.03940836,  0.1224892 ,  0.67077096,
        2.63215434,  1.88716629,  2.08485778,  2.31505959,  0.60206674,
        0.8168441 ,  4.20758822])

In [49]:
feature_importance_series = Series(feature_importances, index=X.columns.values)
feature_importance_series[:10]

Q("age_binned_(-0.001, 78.0]")      0.076504
Q("age_binned_(78.0, 306.0]")       0.771781
Q("age_binned_(306.0, 669.0]")      0.009460
Q("age_binned_(669.0, 1222.0]")     0.255083
Q("age_binned_(1222.0, 9137.0]")    0.469194
Q("animal_type_0")                  1.887166
Q("animal_type_Bird")               0.343172
Q("animal_type_Cat")                0.039408
Q("animal_type_Dog")                0.122489
Q("animal_type_Livestock")          0.670771
dtype: float64

In [50]:
feature_importance_series.sort_values(ascending=False)[:10]

Q("sex_upon_outcome_Unknown")          4.207588
Q("animal_type_Other")                 2.632154
Q("sex_upon_outcome_Intact Male")      2.315060
Q("sex_upon_outcome_Intact Female")    2.084858
Q("sex_upon_outcome_0")                1.887166
Q("animal_type_0")                     1.887166
Q("sex_upon_outcome_Spayed Female")    0.816844
Q("age_binned_(78.0, 306.0]")          0.771781
Q("animal_type_Livestock")             0.670771
Q("sex_upon_outcome_Neutered Male")    0.602067
dtype: float64

In [51]:
top_10_feature_indices = feature_importance_series.sort_values(ascending=False)[:10].index.values

In [52]:
inter_class_differences = model.feature_log_prob_[1] - model.feature_log_prob_[0]
new_feature_importance_series = Series(inter_class_differences, index=X.columns.values)

new_feature_importance_series[top_10_feature_indices]

Q("sex_upon_outcome_Unknown")         -4.207588
Q("animal_type_Other")                -2.632154
Q("sex_upon_outcome_Intact Male")     -2.315060
Q("sex_upon_outcome_Intact Female")   -2.084858
Q("sex_upon_outcome_0")               -1.887166
Q("animal_type_0")                    -1.887166
Q("sex_upon_outcome_Spayed Female")    0.816844
Q("age_binned_(78.0, 306.0]")          0.771781
Q("animal_type_Livestock")            -0.670771
Q("sex_upon_outcome_Neutered Male")    0.602067
dtype: float64